In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import cmath
plt.style.use('seaborn-v0_8-paper')

In [2]:
db = pd.read_csv('saida_motor.csv')

#extracting velocity in rpm from db
rpm = db['Data_4']

#extracting Id and Iq from db
i_d = db['Data_1']
i_q = db['Data_2']

#extracting estimated torque
T_ref = db['Data_5']

print(db)

             Time        Data_1        Data_2        Data_3      Data_4  \
0           0 sec  0.000000e+00  0.000000e+00  0.000000e+00    0.000000   
1       1e-05 sec -8.062316e-20 -3.209883e-35 -2.823443e-19   -0.046356   
2       2e-05 sec -1.809906e-12  6.223484e-06 -4.235165e-19   -0.092712   
3       3e-05 sec -1.266097e-11  1.866093e-05 -4.235165e-19   -0.139067   
4       4e-05 sec -4.518647e-11  3.730283e-05 -4.223871e-19   -0.185423   
...           ...           ...           ...           ...         ...   
599996      6 sec  2.668697e-01  6.740604e+00  6.666304e-09  600.076118   
599997      6 sec  7.105145e-02  6.972852e+00  3.002559e-09  600.082381   
599998      6 sec -1.213717e-01  7.206377e+00  3.002549e-09  600.090416   
599999      6 sec -9.384326e-02  7.114325e+00  2.344110e-11  600.097759   
600000      6 sec -6.725939e-02  7.023140e+00  2.344436e-11  600.104410   

              Data_5  
0      -4.184288e-35  
1       1.018847e-05  
2       3.054981e-05  
3      

In [3]:
#estimated torque equation
def est_torque(Ld, Lq, yf, rpm, i_d, i_q):

    #creating estimated torque equation
    T = (3/4) * 12 * i_q * (Ld * i_d + yf - Lq * i_d)

    #saving db on a csv file
    path = 'torque_est_SA_Original.csv'
    df = pd.DataFrame({
        'rpm': rpm,
        'T_est': T,
    })
    
    df.to_csv(path, index=False)
    
    return path

In [4]:
#root mean square error equation
def RMSE(rpm, T_ref, est_db):

    T_est = pd.read_csv(est_db)

    #T_ref = db['Data_5']
    est = T_est['T_est']

    i = (T_ref - est)**2

    #n = len(db)
    n = len(rpm)

    rmse = np.sqrt((1/n) * np.sum(i))

    #x = db['Data_3']
    error = np.sqrt((T_ref - est)**2)
    
    return rmse

In [5]:
#starting optimazation
from scipy.optimize import dual_annealing
from numpy.random import rand
import time 

#defining parameters inicial values
Ld_init = 0.0066
Lq_init = 0.0066
yf_init = 0.11546

seconds_inicio = time.time()

#setting path
t_est_path = 'torque_est_SA.csv'

def objective(params):
    Ld, Lq, yf = params
    T_est_path = est_torque(Ld, Lq, yf, rpm, i_d, i_q)
    return RMSE(rpm, T_ref, t_est_path)

# define range for input
r_min, r_max = 0.5, 1.5
# define the bounds on the search
bounds = [[r_min*Ld_init, r_max*Ld_init], [r_min*Lq_init, r_max*Lq_init], [r_min*yf_init, r_max*yf_init]]
# perform the dual annealing search
for i in range(30):
    n = i+1
    result = dual_annealing(objective, bounds, no_local_search=True, seed=n, maxiter=1000)
    # summarize the result
    print(f"seed:{n}")
    print('Status : %s' % result['message'])
    print('Total Evaluations: %d' % result['nfev'])

    # evaluate solution
    solution = result['x']
    evaluation = objective(solution)
    print('Solution: f(%s) = %.10f' % (solution, evaluation))
    seconds_final = time.time()
    tempo_total = (seconds_final - seconds_inicio)/3600
    with  open("T_est_SA_Original.txt", "a") as file1:
        file1.write(f"{solution},{evaluation}\n")

FileNotFoundError: [Errno 2] No such file or directory: 'torque_est_SA.csv'